# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [156]:
# Imports here
import os
from collections import OrderedDict

import numpy as np
import torch
import torch.nn.functional as tf
import torchvision as tv

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [157]:
data_dir = 'flowers'
names = ['train', 'valid', 'test']

dataset = {
    name: os.path.join(data_dir, name) for name in names
}


In [158]:
crop_height = 224
crop_width  = 224

image_means  = [0.485, 0.456, 0.406]
image_stdevs = [0.229, 0.224, 0.225]

In [159]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train': tv.transforms.Compose([
                tv.transforms.RandomRotation(20),
                tv.transforms.Resize(256),
                tv.transforms.RandomResizedCrop(crop_width),
                tv.transforms.RandomHorizontalFlip(0.5),
                tv.transforms.ToTensor(),
                tv.transforms.Normalize(image_means, image_stdevs)
    
    ]),

    'valid': tv.transforms.Compose([
                    tv.transforms.Resize(256),
                    tv.transforms.CenterCrop(crop_width),
                    tv.transforms.ToTensor(),
                    tv.transforms.Normalize(image_means, image_stdevs)
                ]),
    
    'test': tv.transforms.Compose([
                tv.transforms.Resize(256),
                tv.transforms.CenterCrop(crop_width),
                tv.transforms.ToTensor(),
                tv.transforms.Normalize(image_means, image_stdevs)    
            ])
}

# TODO: Load the datasets with ImageFolder
image_datasets = {
    name : tv.datasets.ImageFolder(dataset[name], 
                                   transform=data_transforms[name]) for name in names
}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=32, shuffle=True),
    'test':  torch.utils.data.DataLoader(image_datasets['test'], batch_size=32)
}    

def get_batch_size():
    images, _ = next(iter(dataloaders['train']))
    return images.shape[0]


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [160]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [161]:
# Load model pre-trained on ImageNet
model = tv.models.vgg19(pretrained=True)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [162]:
# Freeze pre-trained model
for param in model.parameters():
    param.requires_grad = False

In [163]:
# Define network to use as a classifier.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ClassifierNet(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_prob=0.3):
        """
        """
        super().__init__()
        layers = [input_size] + hidden_layers + [output_size]
        self.layers = torch.nn.ModuleList()
        for x, y in zip(layers[:-1], layers[1:]):
            self.layers.append(torch.nn.Linear(x, y))
        self.dropout = torch.nn.Dropout(p=drop_prob)
    
    def forward(self, x):
        """
        """
        for linear in self.layers[:-1]:
            x = tf.relu(linear(x))
            x = self.dropout(x)
        x = self.layers[-1](x)
        return tf.log_softmax(x, dim=1)


def make_classifier_net(input_size, output_size, hidden_layers, drop_prob=0.3):
    """
    Return instance of torch.nn.Sequential class
    """
    layers = [input_size] + hidden_layers + [output_size]
    input_output_size = [] 
    for x, y in zip(layers[:-1], layers[1:]):
        input_output_size.append((x, y))
        
    modules = []
    for i, (inp_size, out_size) in enumerate(input_output_size[:-1]):
        modules.append(("fc{}".format(i), torch.nn.Linear(inp_size, out_size)))
        modules.append(("relu{}".format(i), torch.nn.ReLU()))
        modules.append(("dropout{}".format(i), torch.nn.Dropout(drop_prob)))
    # Add output layer
    inp_size, out_size = input_output_size[-1]
    modules.append(("fc-output", torch.nn.Linear(inp_size, out_size)))
    modules.append(("logsoftmax", torch.nn.LogSoftmax(dim=1)))
    
    seq = torch.nn.Sequential(OrderedDict(modules))
    return seq   

#model.classifier = ClassifierNet(25088, 102, [4096, 2048, 2048], 0.3)
model.classifier = make_classifier_net(25088, 102, [4096, 2048, 2048], 0.3)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [164]:
# Define validation and training functions.

def validate(model, device, dataloader, criterion):
    """
    Return pair (loss, accuracy) per validation set.
    Loss and accuracy are averaged over validation batches.
    """
    loss = 0
    accuracy = 0
    dataloader = iter(dataloader)
    N = len(dataloader)
    
    model.eval()
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_hat = model.forward(x)
            loss += criterion(y_hat, y).item()

            probs = torch.exp(y_hat)
            accuracy += (y.data == probs.max(dim=1)[1]).type(torch.FloatTensor).mean()
    
    return float(loss) / N, accuracy / N

def train(model, device, dataloader, criterion, optimizer, progress_tracker):
    """
    Return 
        mean train loss
    """
    train_loss = []
    for x, y in iter(dataloader):
        model.train()
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        progress_tracker(loss.item())
    return np.mean(train_loss)
            
        

In [170]:
_CHECKPOINT_DIR = "checkpoints"
def make_checkpoint(model, optimizer, lr, epoch, best_score, perf_report, filename):
    checkpoint = {
        'epoch': epoch,
        'best-score': best_score,
        'optimizer.state_dict': optimizer.state_dict(),
        'lr.state_dict': lr.state_dict(),
        'classifier.state_dict': model.classifier.state_dict(),
        'class_to_idx': image_datasets['train'].class_to_idx,
        'perf_report': perf_report
    }
    torch.save(checkpoint, os.path.join(_CHECKPOINT_DIR, filename))

def list_checkpoints():
    """Return collection of files in checkpoint directory.
    """
    acc = []
    for name in os.listdir(_CHECKPOINT_DIR):
        acc.append(name)
    return acc

def get_checkpoint(filename):
    path = os.path.join(_CHECKPOINT_DIR, filename)
    return torch.load(path)


In [166]:
class ProgressTracker(object):
    def __init__(self, model, device, dataloader, criterion, lr_scheduler, report_every_n):
        """
            Return function that tracks training performance.

            @model       ::= model
            @device      ::= device (cuda or cpu)
            @dataloader  ::= validation data loader
            @criterion   ::= loss criterion
            @lr_scheduler::= learning rate scheduler
            @report_every_n    ::= defines number of training batches between validation tests.
        """
        self.model = model
        self.device = device
        self.dataloader = dataloader
        self.criterion = criterion
        self.lr_scheduler = lr_scheduler
        self.report_every_n = report_every_n
        self._perf_report = []
        self._acc = []
        self._num_iter = 0
        self._epoch = 1
        
    def set_epoch(self, epoch):
        self._epoch = epoch
        
    def performance_report(self):
        return self._perf_report
    
    def __call__(self, train_loss):
        """
        Return 
            - True to continue training.
            - False to stop training, when validation loss stops decreasing.
            
            Criterion for stopping training:
            - Average of net change of validation loss for the most recent @num_samples
            is greater or equal zero.
            
        Record performance metrics (train-loss, validation-loss, validation-accuracy).
        """
        # Adjust learning rate
        self.lr_scheduler.step()
        
        self._acc.append(train_loss)
        n = len(self._acc)
        if n % self.report_every_n == 0:
            self._num_iter = self._num_iter + n
            valid_loss, valid_acc = validate(self.model, self.device, iter(self.dataloader), self.criterion)
            self._perf_report.append((self._num_iter, np.mean(self._acc), valid_loss, valid_acc))
            self._acc = []
            print("epoch: {} n_iter: {}.. ".format(self._epoch, self._num_iter),
                  "Training Loss: {:.3f}.. ".format(self._perf_report[-1][1]),
                  "Test Loss: {:.3f}.. ".format(self._perf_report[-1][2]),
                  "Test Accuracy: {:.3f}".format(self._perf_report[-1][3]))
            
def train_network(model, device):   

    max_lr = 0.02
    
    model.to(device)
    criterion = torch.nn.NLLLoss()
    optimizer = torch.optim.SGD(model.classifier.parameters(), lr=max_lr, momentum=0.8)

    num_batches = len(iter(dataloaders['train'])) # number of batches per epoch
    cycle = 4 # one cycle is 4 epochs
    T = cycle * num_batches
    print("n={} num-iter-per-epoch={} T={}".format(num_batches*get_batch_size(), num_batches, T))
    
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T)
    progress_tracker = ProgressTracker(model, 
                                       device, 
                                       dataloaders['valid'], 
                                       criterion, 
                                       lr_scheduler, 
                                       30)
        
    for epoch_no in range(1, 3*T+1):
        progress_tracker.set_epoch(epoch_no)
        train_loss = train(model, device, dataloaders['train'], criterion, optimizer, progress_tracker)
        if epoch_no % cycle == 0:
            valid_loss, valid_acc = validate(model, device, dataloaders['valid'], criterion)
            make_checkpoint(model, 
                            optimizer, 
                            lr_scheduler, 
                            epoch_no, 
                            {'train-loss': train_loss, 'valid-loss': valid_loss, 'valid-acc': valid_acc},
                            progress_tracker.performance_report(),
                           "checkpoint_epoch_{}.pth".format(epoch_no))
            optimizer = torch.optim.SGD(model.classifier.parameters(), lr=max_lr, momentum=0.8)
            lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T)
            progress_tracker = ProgressTracker(model, 
                                               device, 
                                               dataloaders['valid'], 
                                               criterion, 
                                               lr_scheduler, 
                                               30)            
    return progress_tracker.performance_report()

In [167]:
performance_report = train_network(model, device)

n=6592 num-iter-per-epoch=103 T=412
epoch: 1 n_iter: 30..  Training Loss: 4.554..  Test Loss: 4.417..  Test Accuracy: 0.092
epoch: 1 n_iter: 60..  Training Loss: 4.367..  Test Loss: 4.049..  Test Accuracy: 0.095
epoch: 1 n_iter: 90..  Training Loss: 3.936..  Test Loss: 3.283..  Test Accuracy: 0.191
epoch: 2 n_iter: 120..  Training Loss: 3.353..  Test Loss: 2.647..  Test Accuracy: 0.306
epoch: 2 n_iter: 150..  Training Loss: 2.879..  Test Loss: 2.157..  Test Accuracy: 0.400
epoch: 2 n_iter: 180..  Training Loss: 2.362..  Test Loss: 1.735..  Test Accuracy: 0.514
epoch: 3 n_iter: 210..  Training Loss: 2.056..  Test Loss: 1.495..  Test Accuracy: 0.564
epoch: 3 n_iter: 240..  Training Loss: 1.864..  Test Loss: 1.297..  Test Accuracy: 0.630
epoch: 3 n_iter: 270..  Training Loss: 1.690..  Test Loss: 1.151..  Test Accuracy: 0.661
epoch: 3 n_iter: 300..  Training Loss: 1.603..  Test Loss: 1.061..  Test Accuracy: 0.672
epoch: 4 n_iter: 330..  Training Loss: 1.337..  Test Loss: 0.989..  Test Accu

KeyboardInterrupt: 

In [4]:
# Plot curves
def plot_validation_accuracy(performance_report):
    x = [i for i in range(len(performance_report))]
    train_loss = [row[1] for row in performance_report]
    valid_loss = [row[2] for row in performance_report]
    valid_accuracy = [row[3] for row in performance_report]

    fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
    ax1.set_yscale('log')
    ax1.plot(x, train_loss, label="train_loss")
    ax1.plot(x, valid_loss, label="valid_loss")
    ax1.legend()

    ax2.plot(x, valid_accuracy, label="validation_accuracy")
    ax2.legend()
    plt.show()

#plot_validation_accuracy(performance_report)
torch.optim.lr_scheduler.

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [168]:
# TODO: Do validation on the test set
_, accuracy = validate(model, device, iter(dataloaders['test']), torch.nn.NLLLoss())
print("Test accuracy: {0:.03f}".format(accuracy))

Test accuracy: 0.824


In [174]:
cp = get_checkpoint('checkpoint_epoch_8.pth')
clf = make_classifier_net(25088, 102, [4096, 2048, 2048], 0.3)

newmod = tv.models.vgg19(pretrained=True) 
for param in newmod.parameters():
    param.requires_grad = False
newmod.classifier = clf
newmod.to(device)
newmod.classifier.load_state_dict(cp['classifier.state_dict'])
_, accuracy = validate(newmod, device, iter(dataloaders['test']), torch.nn.NLLLoss())
print("Test accuracy: {0:.03f}".format(accuracy))

Test accuracy: 0.812


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
checkpoint = {
    'optimizer.state_dict': optimizer.state_dict(),
    'classifier.state_dict': model.classifier.state_dict(),
    'class_to_idx': image_datasets['train'].class_to_idx,
    'performance_report': performance_report
}
torch.save(checkpoint, "checkpoint.pth")

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes